# Data Cleaning

**Autor:** Cristian Andrés Zapata Arenas  
**Objetivo:** transformar los datos crudos en una fuente confiable para la extracción de insights profundos, eliminando inconsistencias, duplicados y datos faltantes.

La tercera entrega se centrará en la **limpieza y transformación de datos**, la fase más crítica para garantizar la fiabilidad de nuestro análisis. Utilizando **Pandas** y las conclusiones obtenidas del EDA, abordaremos los desafíos identificados: imputaremos valores faltantes, corregiremos inconsistencias en el formato de las variables categóricas y estandarizaremos las unidades. Nuestro objetivo es refinar el *dataset* de estilo de vida y sueño, dejándolo **pulido, estructurado y listo** para la fase final de modelado predictivo, asegurando que cada *insight* se base en datos de máxima calidad.

## Limpieza Transversal (Duplicados y Nombres)

### Problema # 1: Eliminación de registros duplicados

Se eliminan las filas que son réplicas exactas en todas sus columnas. Esto asegura que cada observación sea única y evita sesgos en el análisis estadístico.



In [1]:
import pandas as pd # type: ignore

# Se carga el archivo en formato CSV
data = pd.read_csv('C:/Users/crisz/Downloads/Sleep_health_and_lifestyle_dataset.csv',
                   index_col="Person ID")

# Asignamos el DataFrame a una variable descriptiva.
dfLifestyle = data
    
# 1.3. Problema #1: Eliminación de Registros Duplicados
print("\n--- 1. Eliminación de Registros Duplicados ---")

# Almacenar el conteo inicial.
num_filas_antes = len(dfLifestyle)

# Se eliminan filas idénticas en todas las columnas, manteniendo la primera aparición.
dfLifestyle.drop_duplicates(inplace=True)

# Reporte de la operación.
num_duplicados_eliminados = num_filas_antes - len(dfLifestyle)
print(f"Se encontraron y eliminaron {num_duplicados_eliminados} filas duplicadas. Filas restantes: {len(dfLifestyle)}")


--- 1. Eliminación de Registros Duplicados ---
Se encontraron y eliminaron 242 filas duplicadas. Filas restantes: 132


### Problema # 2: Estandarización de nombres y columnas

Se abordan las problemáticas de nomenclatura y el formato de datos compuestos para preparar las variables. Se traduce y se aplica el formato snake_case a todos los nombres de columna para facilitar la programación y la legibilidad.

In [2]:
# Problema #2: Estandarización y Traducción de Nombres
print("\n--- 2. Estandarización de Nombres de Columnas ---")

# Diccionario de mapeo de nombres.
mapeo_nombres = {
    'Gender': 'genero', 'Age': 'edad', 'Occupation': 'ocupacion',
    'Sleep Duration': 'duracion_sueno_horas', 'Quality of Sleep': 'calidad_sueno_escala_1_10',
    'Physical Activity Level': 'actividad_fisica_minutos_dia', 'Stress Level': 'nivel_estres_escala_1_10',
    'BMI Category': 'categoria_imc', 'Blood Pressure': 'presion_sanguinea_sistolica_diastolica',
    'Heart Rate': 'ritmo_cardiaco_bpm', 'Daily Steps': 'pasos_diarios', 'Sleep Disorder': 'trastorno_sueno'
}

dfLifestyle.rename(columns=mapeo_nombres, inplace=True)
print("Nombres de columnas estandarizados a 'snake_case' y traducidos.")


--- 2. Estandarización de Nombres de Columnas ---
Nombres de columnas estandarizados a 'snake_case' y traducidos.


### Problema # 3: Separación de la Columna Compuesta (presion_sanguinea...)

La columna de presión, con formato sistólica/diastólica, se divide en dos columnas numéricas para permitir cálculos estadísticos.

In [3]:
# Problema #3: Separación de Presión Sanguínea
print("\n--- 3. Separación de la Columna Compuesta de Presión Sanguínea ---")

columna_compuesta = 'presion_sanguinea_sistolica_diastolica'

# División de la cadena por el caracter '/'.
bp_series = dfLifestyle[columna_compuesta].str.split('/', expand=True)

# Creación de las nuevas columnas y conversión a tipo entero (`int`).
dfLifestyle['presion_sistolica'] = bp_series[0].astype(int)
dfLifestyle['presion_diastolica'] = bp_series[1].astype(int)

# Eliminación de la columna original.
dfLifestyle.drop(columna_compuesta, axis=1, inplace=True)

print("Columna de Presión Sanguínea dividida y convertida a tipo entero.")


--- 3. Separación de la Columna Compuesta de Presión Sanguínea ---
Columna de Presión Sanguínea dividida y convertida a tipo entero.


## Limpieza de Valores categóricos y numéricos

### Problema # 4: Tratamiento de Datos Faltantes (trastorno_sueno)

Se imputan los valores nulos (NaN) en trastorno_sueno con la categoría None (Ninguno), ya que la ausencia de un registro de trastorno implica su inexistencia en la observación.

In [5]:
# Problema #4: Imputación de Valores Faltantes (¡SOLUCIÓN A LA FUTURE WARNING!)
print("\n--- 4. Imputación de Valores Faltantes en 'trastorno_sueno' ---")

columna_nulos = 'trastorno_sueno'

# CORRECCIÓN: Se usa asignación directa para evitar la FutureWarning y el chained assignment.
dfLifestyle[columna_nulos] = dfLifestyle[columna_nulos].fillna('None')

print(f"Valores faltantes en '{columna_nulos}' rellenados con 'None' (sin advertencias).")


--- 4. Imputación de Valores Faltantes en 'trastorno_sueno' ---
Valores faltantes en 'trastorno_sueno' rellenados con 'None' (sin advertencias).


## Inconsistencia Categórica

### Problema # 5 y 6: Inconsistencia Categórica

In [6]:
# Problema #5 y #6: Unificación y Estandarización de Categorías
print("\n--- 5. y 6. Unificación y Estandarización de Variables Categóricas ---")

# a) Problema #5: Unificación de 'categoria_imc'
dfLifestyle['categoria_imc'] = dfLifestyle['categoria_imc'].replace('Normal Weight', 'Normal')
print("-> Categorías de IMC: 'Normal Weight' unificada a 'Normal'.")

# b) Problema #6: Estandarización de 'trastorno_sueno'
columna_trastorno = 'trastorno_sueno'
dfLifestyle[columna_trastorno] = dfLifestyle[columna_trastorno].str.lower().str.strip()
mapeo_trastornos = {'sleep apnea': 'apnea del sueno', 'insomnia': 'insomnio', 'none': 'ninguno'}
dfLifestyle[columna_trastorno] = dfLifestyle[columna_trastorno].replace(mapeo_trastornos)

print("-> Categorías de Trastorno del Sueño: Estandarizadas y traducidas.")
print(f"Valores únicos finales: {dfLifestyle[columna_trastorno].unique()}")
print("Categorías corregidas.")


--- 5. y 6. Unificación y Estandarización de Variables Categóricas ---
-> Categorías de IMC: 'Normal Weight' unificada a 'Normal'.
-> Categorías de Trastorno del Sueño: Estandarizadas y traducidas.
Valores únicos finales: ['ninguno' 'apnea del sueno' 'insomnio']
Categorías corregidas.


### Problema # 7: Ajuste de Granularidad Numérica

In [7]:
# Problema #7: Ajuste de Granularidad Numérica
print("\n--- 7. Ajuste de Granularidad en 'duracion_sueno_horas' ---")

columna_granularidad = 'duracion_sueno_horas'

# Se aplica un redondeo a un decimal para consistencia.
dfLifestyle[columna_granularidad] = dfLifestyle[columna_granularidad].round(1)

print(f"Columna '{columna_granularidad}' redondeada a un decimal.")


--- 7. Ajuste de Granularidad en 'duracion_sueno_horas' ---
Columna 'duracion_sueno_horas' redondeada a un decimal.


## Verificación Final

In [41]:
# Verificación Final del Estado del DataFrame
print("\n=======================================================")
print("  VERIFICACIÓN FINAL: DATAFRAME LISTO PARA ANÁLISIS")
print("=======================================================\n")

# Se verifica que no queden valores nulos y que los tipos de datos sean los esperados.
dfLifestyle.info()

print("\n--- Muestra Final del DataFrame Limpio ---\n")
# Muestra de las primeras filas con las columnas ya limpias y transformadas.
print(dfLifestyle.head())

print("\nEl proceso de limpieza integral (7 problemáticas) ha sido completado con éxito y sin advertencias.")


  VERIFICACIÓN FINAL: DATAFRAME LISTO PARA ANÁLISIS

<class 'pandas.core.frame.DataFrame'>
Index: 132 entries, 1 to 367
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   genero                        132 non-null    object 
 1   edad                          132 non-null    int64  
 2   ocupacion                     132 non-null    object 
 3   duracion_sueno_horas          132 non-null    float64
 4   calidad_sueno_escala_1_10     132 non-null    int64  
 5   actividad_fisica_minutos_dia  132 non-null    int64  
 6   nivel_estres_escala_1_10      132 non-null    int64  
 7   categoria_imc                 132 non-null    object 
 8   ritmo_cardiaco_bpm            132 non-null    int64  
 9   pasos_diarios                 132 non-null    int64  
 10  trastorno_sueno               132 non-null    object 
 11  presion_sistolica             132 non-null    int64  
 12  presion_diastol